In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

in_dir = "/Users/hn/Desktop/Desktop/Kirti/check_point/analogs/"
out_dir = "/Users/hn/Desktop/"

In [2]:
all_data_usa = pd.read_csv(in_dir + "all_data_usa.csv") 
avg_rcp45 = pd.read_csv(in_dir + "averaged_data_rcp45.csv") 
all_data_usa.head(2)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario
0,1979,32.46875_-109.90625,68,2.939262,1.319697,7.080458,1.348167,0.0,300.525,4821.668922,0,observed
1,1979,32.46875_-109.96875,68,2.946050,1.245847,7.104994,1.396980,0.0,303.925,4831.770062,0,observed


In [24]:
for ii in range(1,avg_rcp45.shape[0]):
    print (ii)

1
2
3
4
5
6
7
8
9


In [3]:
avg_rcp45.head(2)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario
0,2026,43.53125_-116.59375,92,2.352305,23.368053,27.85959,3.009202,0.025247,268.491667,3669.118839,1,ensembe_mean
1,2026,43.59375_-116.78125,92,2.336201,24.035184,27.50817,2.912981,0.027137,260.750000,3623.723188,1,ensembe_mean


## Append

  - First append the needed treated unit to all historical/control units so we can find nearest locations.
  - Then take numeric part of the data to be able to operate on them.
        - normalize
        - find distances
  - add the distances back to the data frame.
  - sort the data frame according to distances.

In [4]:
one_site = avg_rcp45.iloc[0, :].copy()
dt = all_data_usa.append(one_site)
dt['year'] = dt['year'].apply(str);
dt['treatment'] = dt['treatment'].apply(str);
dt.head(2)
dt_orig = dt.copy()

## Normalization

In [5]:
mins = dt.min(axis=0, numeric_only=True)
maxs = dt.max(axis=0, numeric_only=True)
dt.iloc[:, 2:10]  = (dt.iloc[:, 2:10] - mins) / (maxs - mins)
dt.head(2)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario
0,1979,32.46875_-109.90625,0.312102,0.664839,0.015029,0.107926,0.065749,0.0,0.100978,0.632489,0,observed
1,1979,32.46875_-109.96875,0.312102,0.666985,0.014188,0.108300,0.068130,0.0,0.102268,0.634309,0,observed


### Compute euclidean distances
Subtract the last row from all other rows to compute distances.


In [6]:
dt.iloc[:, 2:10] = dt.iloc[:, 2:10] - dt.iloc[-1, 2:10]
print (dt.head(2))
distances = (np.sqrt((dt.iloc[:, 2:10]**2).sum(axis=1))).values.reshape((-1,1))

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario
0,1979,32.46875_-109.90625,-0.152866,0.185546,-0.251089,-0.316732,-0.0810072,-0.00678537,0.012149,0.207621,0,observed
1,1979,32.46875_-109.96875,-0.152866,0.187692,-0.25193,-0.316358,-0.0786266,-0.00678537,0.0134385,0.20944,0,observed


In [8]:
dt_orig['distances'] = distances
dt_orig = dt_orig.sort_values(by=['distances'])
dt_orig.head(3)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario,distances
0,2026,43.53125_-116.59375,92,2.352305,23.368053,27.859590,3.009202,0.025247,268.491667,3669.118839,1,ensembe_mean,0.000000
37049,2007,43.53125_-116.78125,96,2.344302,26.196502,31.237424,2.176265,0.000000,185.725000,3513.329901,0,observed,0.088392
20240,1994,43.53125_-116.78125,99,2.284805,25.017968,27.828322,1.544439,0.000000,220.825000,3547.658606,0,observed,0.093547


In [9]:
out_name = one_site['location'] + '_' + str(one_site['year']) + "_1.csv"
print (out_name)

dt_orig.to_csv(out_dir + out_name, sep='\t')

43.53125_-116.59375_2026_1.csv


In [13]:
dt_orig.head(10)

,year,location,medianDoY,NumLarvaGens_Aug,mean_escaped_Gen1,mean_escaped_Gen2,mean_escaped_Gen3,mean_escaped_Gen4,mean_precip,mean_gdd,treatment,ClimateScenario,distances
0,2026,43.53125_-116.59375,92,2.352305,23.368053,27.859590,3.009202,0.025247,268.491667,3669.118839,1,ensembe_mean,0.000000
37049,2007,43.53125_-116.78125,96,2.344302,26.196502,31.237424,2.176265,0.000000,185.725000,3513.329901,0,observed,0.088392
20240,1994,43.53125_-116.78125,99,2.284805,25.017968,27.828322,1.544439,0.000000,220.825000,3547.658606,0,observed,0.093547
37055,2007,43.59375_-116.84375,96,2.327640,26.807480,31.149851,2.006078,0.000000,181.925000,3490.173473,0,observed,0.096518
17659,1992,43.59375_-116.78125,91,2.331342,20.596046,31.195072,1.578679,0.000000,188.325000,3526.934183,0,observed,0.100795
17654,1992,43.53125_-116.78125,91,2.373916,19.548152,32.109769,1.904073,0.000000,181.250000,3572.069306,0,observed,0.102606
17660,1992,43.59375_-116.84375,92,2.328483,20.528900,31.392022,1.560058,0.000000,175.000000,3510.915864,0,observed,0.105411
17665,1992,43.65625_-116.84375,92,2.297203,21.488987,30.851570,1.373421,0.000000,189.175000,3467.119831,0,observed,0.107147
37054,2007,43.59375_-116.78125,97,2.283699,27.868875,30.023680,1.683234,0.000000,192.650000,3440.658468,0,observed,0.109310
20245,1994,43.59375_-116.78125,100,2.256775,25.916507,27.315955,1.355569,0.000000,227.075000,3510.690660,0,observed,0.109683


In [25]:
A = ["a", "b"]

In [27]:
for a in A:
    print(a)

a
b
